In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [4]:
import os
os.chdir("..")

In [5]:
with open('intents/intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 5)                 85        
                                                                 
Total params: 16629 (64.96 KB)
Trainable params: 16629 (64.96 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [9]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
1/1 [==============================] - 0s 453ms/step - loss: 1.6103 - accuracy: 0.2105
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6095 - accuracy: 0.2105
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6087 - accuracy: 0.2105
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6081 - accuracy: 0.2105
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6076 - accuracy: 0.2105
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 1.6071 - accuracy: 0.2105
Epoch 7/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6066 - accuracy: 0.2105
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 1.6061 - accuracy: 0.2105
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6055 - accuracy: 0.2105
Epoch 10/500
1/1 [==============================] - 0s 2ms/step - loss: 1.6050 - accuracy: 0.2632
Epoch 11/500
1/1 [=========

In [12]:
# to save the trained model
model.save("./models/chat_model")

import pickle

# to save the fitted tokenizer
with open('./models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('./models/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: ./models/chat_model\assets


INFO:tensorflow:Assets written to: ./models/chat_model\assets
